# Config

In [88]:
# Import dependencies
import pandas as pd
from datetime import datetime
from sleeperpy import User, Leagues
from ffpackage.viz import compareFranchises
import plotly
import plotly.express as px

In [89]:
# Settings
date_string = datetime.today().strftime('%Y%m%d')
weekStart = 1
weekEnd = 18
def_starting_pts = 18
pr_bonus = 1
kr_bonus = 3
username = 'dirtywizard'

# Extract

In [90]:
# Read database
preds = pd.read_csv('db/records_preds.csv')
preds = preds.loc[preds['date_prediction']==preds['date_prediction'].max()]
preds = preds.loc[(preds['week']>=weekStart)&(preds['week']<=weekEnd)]
mults = pd.read_csv('db/pts_multipliers.csv')
rostLims = pd.read_csv('db/roster_limits.csv')

In [91]:
def processRosters(league_id):
    # Get rosters for all league members
    rosters = pd.DataFrame(Leagues.get_rosters(league_id))
    # Explode the player columns to find how they are placed on the roster
    result = pd.DataFrame(columns=['owner_id', 'player_id'])
    for col in ['taxi', 'starters', 'players']:
        df = rosters[['owner_id', col]].explode(col)
        df = df.rename(columns={col:'player_id'})
        df[col] = True
        result = result.merge(df, how='outer', on=['owner_id', 'player_id'])
    return result

In [92]:
def procesOwners(league_id):
    owners = pd.DataFrame(Leagues.get_users(league_id))
    list = []
    for i in range(len(owners)):
        if 'team_name' in owners['metadata'][i]:
            list.append(owners['metadata'][i]['team_name'])
        else:
            list.append(owners['display_name'][i])
    owners['franchise_name'] = list
    owners = owners[['user_id', 'franchise_name', 'display_name']]
    return owners

In [93]:
# Get User's Sleeper ID
user_id = User.get_user(username)['user_id']
# Get the User's Sleeper league ID
league_id = Leagues.get_all_leagues(user_id, 'nfl', 2023)[0]['league_id']
# Get all players on rosters
rosters = processRosters(league_id)
# Get info on all owners
owners = procesOwners(league_id)
# Get roster positions 
rosterList = Leagues.get_league(league_id)['roster_positions']

# Transform

In [94]:
# Customize predictions for league
# Calculate projected points according to that league's scoring settings
# Multiply by points multiplier
for col in mults.columns[1:]:
    preds[col] = preds[col] * mults.loc[mults['username']==username, col][0]
# Add up point columns
preds['pts_proj'] = preds[mults.columns[1:]].sum(axis=1)
# Add extras for defense starting values
preds.loc[preds['Yds Allowed'].notna(), 'pts_proj'] = preds.loc[preds['Yds Allowed'].notna(), 'pts_proj'] + def_starting_pts
# Add extras for Punt/kick returners
preds.loc[preds['PR']==True, 'pts_proj'] = preds.loc[preds['PR']==True, 'pts_proj'] + (pr_bonus)
preds.loc[preds['KR']==True, 'pts_proj'] = preds.loc[preds['KR']==True, 'pts_proj'] + (kr_bonus)

# Merge in point projections with franchise data
full = preds \
    .merge(rosters, how='left', on='player_id') \
    .merge(owners, how='left', left_on='owner_id', right_on='user_id')

In [103]:
def getRelProjs(df):
    # Calculate relative points for each position in each week
    df['pts_proj'] = df['pts_proj'].fillna(0)
    df = df.sort_values(by='pts_proj', ascending=False, ignore_index=True)
    # Loop through weeks and positions
    for week in df['week'].unique():
        for pos in rostLims.columns[1:]:
            # Find floor
            cond1 = df['position']==pos
            cond2 = df['week']==week
            floor = df.loc[cond1&cond2, 'pts_proj'].reset_index(drop=True)[rostLims.loc[rostLims['username']==username, pos][0]]
            # Find each player's points over floor
            df.loc[cond1&cond2, 'rel_proj'] = df.loc[cond1&cond2, 'pts_proj'] - floor
    return df

In [104]:
def getRelProjs_optimized(df):
    # Calculate relative points for each position in each week
    df['pts_proj'] = df['pts_proj'].fillna(0)
    df = df.sort_values(by='pts_proj', ascending=False, ignore_index=True)
    # Loop through weeks and positions
    for week in df['week'].unique():
        for pos in df['position'].unique():
            # Find floor
            cond_pos = df['position']==pos
            cond_week = df['week']==week
            floor = df.loc[cond_pos&cond_week, 'pts_proj'].min()
            # Find each player's points over floor
            df.loc[cond_pos&cond_week, 'rel_proj'] = df.loc[cond_pos&cond_week, 'pts_proj'] - floor
    return df

In [105]:
def getSeasonTotals(df):
    # Sum up pts_proj and rel_proj
    totals_proj = df.groupby('player_id')['pts_proj'].sum().reset_index()
    totals_rel = df.groupby('player_id')['rel_proj'].sum().reset_index()
    # Merge in totals with player metadata
    unis = df.drop_duplicates(subset='player_id', ignore_index=True).drop(columns=['pts_proj', 'rel_proj'])
    df = unis.merge(totals_proj, how='left', on='player_id').merge(totals_rel, how='left', on='player_id')
    return df

In [106]:
def tidyUp(df):
    # Tidy
    df = df[[
        'week', 'player_id', 'pts_proj', 'rel_proj', 'franchise_name', 'full_name', 'age', 'years_exp', 'weight',
        'height', 'team', 'position', 'taxi', 'starters', 'players', 'fantasy_positions', 'depth_chart_order',
        'depth_chart_position', 'status', 'injury_status', 'injury_notes',
        'injury_start_date', 'practice_participation', 'practice_description','active'
    ]]
    return df

# Waiver Report

In [107]:
# Waiver report
waivers = full.copy()
# Calculate relative points for each position in each week
waivers = getRelProjs(waivers)
# Sum points across the season
waivers = getSeasonTotals(waivers)
# Filter for only players on user's roster or unclaimed and who would improve user's points
minpts = waivers.loc[(waivers['display_name']==username)&(waivers['taxi'].isna())]['pts_proj'].min()
cond_user = waivers['display_name']==username
cond_fa = waivers['display_name'].isna()
cond_minpts = waivers['pts_proj']>=minpts
waivers = waivers.loc[(cond_user|cond_fa)&cond_minpts].reset_index(drop=True)
# Tidy
waivers = tidyUp(waivers)

# Visualize Total Points

In [110]:
# Summative report
player_sums = full.copy()
# Calculate relative points for each position in each week
player_sums = getRelProjs(player_sums)
# Sum points across the season
player_sums = getSeasonTotals(player_sums)

# Visualize
fig = compareFranchises(player_sums, how='relative')
fig.show()

In [111]:
# Visualize
fig = compareFranchises(player_sums, how='absolute')
fig.show()

# Roster Optimizer

In [121]:
#from ffpackage.analysis import starterSelector
# Roster optimizer
df = full.copy()
optimized = pd.DataFrame()
# Roster Dictionary
rosterDict = {
    'QB':['QB'], 
    'RB':['RB'],
    'WR':['WR'],
    'TE':['TE'],
    'K':['PK'],
    'DEF':['DEF'],
    'FLEX':['RB', 'WR', 'TE'],
    'SUPER_FLEX':['QB', 'RB', 'WR', 'TE'],
}
# Sort
cond_taxi = df['taxi'].isna()
sorted = df.loc[cond_taxi].sort_values(by='pts_proj', ignore_index=True, ascending=False)
# Loop through weeks and franchises
for this_user in df['user_id'].unique():
    for week_to_optimize in df['week'].unique():
        # Filter predictions for only that week and on a roster
        cond_franch = sorted['user_id']==this_user
        cond_week = sorted['week']==week_to_optimize
        week_filtered = sorted.loc[cond_franch & cond_week]
        # Start selecting the most valuable players in terms of total points for each position
        result = []
        for pos in rosterList:
            # Ignore bench spots
            if pos != "BN":
                # Filter for each position and make sure the player is not already taken
                cond_pos = week_filtered['position'].isin(rosterDict[pos])
                cond_nottaken = ~week_filtered['player_id'].isin(result)
                player_filtered = week_filtered.loc[cond_pos & cond_nottaken]
                # Find highest scoring player that fits
                if len(player_filtered)>0:
                    player = player_filtered.reset_index(drop=True)['player_id'][0]
                    result.append(player)
        # Append results
        next = week_filtered.loc[week_filtered['player_id'].isin(result)]
        optimized = pd.concat([optimized, next], axis=0, ignore_index=True)

# Get relative points
optimized = getRelProjs_optimized(optimized)
# Sum points across the season
optimized = getSeasonTotals(optimized)
# Visualize
fig = compareFranchises(optimized, how='absolute')
fig.show()

### Left Off Here:
##### Build trade report
##### RosterOptimizer
##### Tailor scoring for user's unique league

# Write to CSV

In [98]:
# sums.to_csv(f'processed_{date_string}.csv', index=False)
waivers.to_csv(f'waivers_{date_string}.csv', index=False)

In [97]:
# Starter report
full.loc[full['starters']==True].groupby('franchise_name')['rel_pts'].sum().sort_values()

KeyError: 'Column not found: rel_pts'

In [96]:
full.loc[full['starters']==True].groupby('franchise_name')['pts_proj'].sum().sort_values()

franchise_name
DangeRUSS Last Ride          160.0080
Comeback 👑’s                1885.1302
kevinbash                   1925.8060
Idk much about soccer       2105.1885
Count of Monte Christian    2163.8135
2014champ                   2194.1737
GusTheBus                   2283.6228
📜 Providence 🪬 Spirits 🥂    2324.0345
Verdanks Vacqueros FFC      2426.5528
Pretty Big Wieners          2483.3059
Saskatoon Squatches         2582.2908
Croccity Body Snatchers     2733.0872
Name: pts_proj, dtype: float64